In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from sklearn import preprocessing, impute, model_selection, decomposition, cluster

### Import EPA Pollution and Census American Community Survey Data

In [2]:
epa = pd.read_csv('../dataset/epa_pollution.csv')
census = pd.read_csv('../dataset/census_acs.csv')
aqi = pd.read_csv('../dataset/aqi_report.csv')

#only use data with common cbsa codes
common_cbsa = [i for i in epa['cbsa_code'].unique() if (i in census['cbsa_code'].unique()) and (i in aqi['cbsa_code'].unique())]

# Standardized cbsa code datasets
pollution = epa[epa['cbsa_code'].isin(common_cbsa)]
soc_econ = census[census['cbsa_code'].isin(common_cbsa)]
aqi_report = aqi[aqi['cbsa_code'].isin(common_cbsa)]

In [3]:
len(common_cbsa)

339

## Data Preprocessing

### Census ACS 

In [4]:
# Remove redundant/irrelevant columns
soc_econ = soc_econ.drop(['metropolitan_area', 'city', 'state'], axis=1)

# Normalize columns
# Income
income = [i for i in soc_econ.columns if '$' in i]
for col in income:
    soc_econ[col] = soc_econ[col]/soc_econ['Income_Total']
soc_econ.drop('Income_Total', axis=1, inplace=True)

#Education
education = [i for i in soc_econ.columns if ('degree' in i) or ('graduate' in i)]
for col in education:
    soc_econ[col] = soc_econ[col]/soc_econ['Education_Total']
soc_econ.drop('Education_Total', axis=1, inplace=True)

# Occupation
occupation = [i for i in soc_econ.columns if (i not in income) and (i not in education) and (i not in ['cbsa_code', 'year', 'Occupation_Total'])]
for col in occupation:
    soc_econ[col] = soc_econ[col]/soc_econ['Occupation_Total']
soc_econ.drop('Occupation_Total', axis=1, inplace=True)

soc_econ.head()

,cbsa_code,Less than high school graduate,High school graduate (includes equivalency),Some college or associate's degree,Bachelor's degree,Graduate or professional degree,"$45,000 to $49,999","$50,000 to $59,999","$60,000 to $74,999","$75,000 to $99,999",...,"Educational services, and health care and social assistance","Arts, entertainment, and recreation, and accommodation and food services","Other services, except public administration",Public administration,"Management, business, science, and arts occupations",Service occupations,Sales and office occupations,"Natural resources, construction, and maintenance occupations","Production, transportation, and material moving occupations",year
1,10420,0.109590,0.342669,0.266596,0.183831,0.097314,0.043547,0.085722,0.105218,0.116918,...,0.224313,0.087933,0.047411,0.029834,0.341412,0.164147,0.268786,0.001810,0.223845,2005
2,10500,0.214530,0.323630,0.276552,0.115232,0.070055,0.046814,0.099690,0.111882,0.087279,...,0.221430,0.066773,0.066212,0.076664,0.300161,0.138019,0.246350,0.009773,0.305697,2005
3,10580,0.097922,0.308826,0.284550,0.169404,0.139298,0.046200,0.092376,0.111600,0.137878,...,0.259730,0.069128,0.038137,0.110608,0.397041,0.151805,0.270669,0.002259,0.178227,2005
4,10740,0.140155,0.265774,0.293919,0.169377,0.130775,0.043810,0.083659,0.113831,0.102810,...,0.223604,0.094226,0.045499,0.063296,0.374627,0.166710,0.264363,0.002006,0.192294,2005
5,10780,0.176673,0.371108,0.262197,0.136293,0.053729,0.041082,0.072454,0.070793,0.091749,...,0.302493,0.063796,0.038255,0.085138,0.334400,0.170620,0.256644,0.007136,0.231201,2005


In [5]:
soc_econ.shape

(4644, 33)

#### PCA

Since there are 33 attributes and only 5K instances, it may not be sufficient for a successful clustering analysis.

In [6]:
#Capture 90% variance 
pca = decomposition.PCA(n_components=.90, svd_solver='full')

#only apply PCA on the continious variables 
continuous = soc_econ.drop(['cbsa_code', 'year'], axis=1)
reduced_census = pd.DataFrame(pca.fit_transform(continuous))
reduced_census['cbsa_code'] = soc_econ['cbsa_code']
reduced_census['year'] = soc_econ['year']

soc_econ_df = reduced_census.sort_values(['cbsa_code', 'year'], axis=0)
soc_econ_df.shape

(4644, 11)

### Clean AQI Report

In [8]:
aqi_report = aqi.copy()
aqi_report.sort_values(['cbsa_code', 'year'], inplace=True, axis=0)

# Normalize the days for each air quality with the total AQI days
qualities = ['Good', 'Moderate', 'Unhealthy for Sensitive Groups', 'Unhealthy', 'Very Unhealthy']
for quality in qualities:
    aqi_report.loc[:, quality] = aqi_report[quality] / aqi_report['# Days with AQI']
    
aqi_report.drop(columns="# Days with AQI", inplace=True)

### EPA Pollution

Clean up data such that each instance contains all pollutants for each cbsa

In [9]:
durations = ['1 HOUR','24-HR BLK AVG', '24 HOUR', '3-HR BLK AVG', '8-HR RUN AVG END HOUR', '8-HR RUN AVG BEGIN HOUR', '5 MINUTE', 'INTEGRATED PASSIVE 4-WEEKS', 'INTEGREATED PASSIVE 3-WEEKS']
first_duration = {'PM2.5': '1 HOUR',
                  'Ozone': '8-HR RUN AVG END HOUR',
                  'Carbon_monoxide': '1 HOUR',
                  'Sulfur_dioxide': '1 HOUR'}

## Create new pollution DF with all the pollutants for every instance
pollution_df = pd.DataFrame()
for year in range(2005, 2020):
    for code in common_cbsa:
        new_row = {'year': year,
                   'cbsa_code': code}

        #average all records for year and code into one instance
        for k, v in first_duration.items():
            pollutant = pollution[(pollution['year'] == year) & (pollution['cbsa_code'] == code) & (pollution['parameter'].str.contains(k.replace('_', ' ')))]
            avg_poll = pollutant[pollutant['sample_duration'] == v]
            new_row[k + '_sample_duration'] = v
            
            #If no instances matching the sample duration exist, then we look for other
            for duration in durations:
                if avg_poll.shape[0] != 0: break
                avg_poll = pollutant[pollutant['sample_duration'] == duration]
                new_row[k + '_sample_duration'] = duration
            
            if avg_poll.shape[0] == 0: 
                new_row[k + '_sample_duration'] = np.nan
            
            attributes = ['arithmetic_mean', 'standard_deviation', 'ninety_ninth_percentile', 'seventy_fifth_percentile']
            for attr in attributes:
                new_row[k + '_' + attr] = pollutant[attr].mean()

        # Add matching AQI report to index
        aqi_dict = aqi_report[(aqi_report['cbsa_code'] == code) & (aqi_report['year'] == year)]
        for col in aqi_dict.columns:
            if col != 'year' and col != 'cbsa_code':
                new_row[col] = aqi_dict.iloc[0, :][col] if (aqi_dict.shape[0] != 0) else None
                
        pollution_df = pollution_df.append(new_row, ignore_index=True)
        
pollution_df.sort_values(['cbsa_code', 'year'], inplace=True, axis=0)

#### Fill and Impute Missing Values

In [10]:
# Fill nan pollution values with recent year values
for code in common_cbsa:
    pollution_df[pollution_df['cbsa_code'] == code] = pollution_df[pollution_df['cbsa_code'] == code].fillna(method='ffill', axis=0) ## Fill forward to bring last completed year forward
    pollution_df[pollution_df['cbsa_code'] == code] = pollution_df[pollution_df['cbsa_code'] == code].fillna(method='bfill', axis=0) ## Fill backward to impute missing values in earlier years

In [11]:
#Impute sample duration with mode
si = impute.SimpleImputer( strategy='most_frequent')
dur_labels = [col for col in pollution_df.columns if 'duration' in col]
pollution_df[dur_labels] = si.fit_transform(pollution_df[dur_labels])

# Label encode all sample_duration columns
le = preprocessing.LabelEncoder()
le.fit(durations)
for col in dur_labels:
    pollution_df[col] = le.transform(pollution_df[col])

In [12]:
# Fill in remaining values with average
si = impute.SimpleImputer()
pollution_df = pd.DataFrame(si.fit_transform(pollution_df), columns=pollution_df.columns)

In [13]:
pollution_df.sort_values(['cbsa_code', 'year'], inplace=True, axis=0)
pollution_df.to_csv('../dataset/epa_pollution_clean.csv', index=False)

#### PCA

Since there are 33 attributes and only 5K instances, it may not be sufficient for a successful clustering analysis.

In [14]:
#Capture 90% variance 
pca = decomposition.PCA(n_components=.90, svd_solver='full')

#only apply PCA on the continious variables 
continuous = pollution_df.drop(['cbsa_code', 'year'], axis=1)
continuous = continuous.drop(dur_labels, axis=1)
reduced_epa = pd.DataFrame(pca.fit_transform(continuous))
reduced_epa['cbsa_code'] = pollution_df['cbsa_code']
reduced_epa['year'] = pollution_df['year']
for duration in dur_labels:
    reduced_epa[duration] = pollution_df[duration]

reduced_epa.sort_values(['cbsa_code', 'year'], inplace=True, axis=0)
# reduced_epa.to_csv('../dataset/epa_pollution_clean.csv', index=False)
reduced_epa.shape

(5085, 10)

### K-Means clustering on EPA pollution + AQI dataset

In [15]:
kmm = cluster.KMeans(n_clusters=5, random_state=27)
kmm.fit_predict(reduced_epa.drop(columns=['year', 'cbsa_code']))
reduced_epa['cluster'] = kmm.labels_

c:\users\suchi\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:1673: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


### K-Means clustering on AQI

In [16]:
kmm = cluster.KMeans(n_clusters=5, random_state=27)
kmm.fit_predict(aqi_report.drop(columns=['year', 'cbsa_code']))
aqi_report['cluster'] = kmm.labels_

### K-Means clustering on ACS

In [17]:
kmm = cluster.KMeans(n_clusters=5, random_state=27)
kmm.fit_predict(soc_econ_df.drop(columns=['year', 'cbsa_code']))
soc_econ['cluster'] = kmm.labels_

###  Dynamic Time Warping K-Means Clustering

In [ ]:
from tslearn.clustering import TimeSeriesKMeans

model = TimeSeriesKMeans(n_clusters=3, metric="dtw", max_iter=50, random_state=143)
y_pred = model.fit_predict(soc_econ)

In [ ]:
model.cluster_centers_.shape

In [ ]:
for yi in range(3):
    plt.subplot(3, 3, 7 + yi)    
    plt.plot(model.cluster_centers_[yi].ravel(), "r-")
    plt.text(0.55, 0.85,'Cluster %d' % (yi + 1),
             transform=plt.gca().transAxes)
    if yi == 1:
        plt.title("DTW $k$-means")

plt.tight_layout() 
plt.show()